In [ ]:
# here we write out different versions of existing heatmap functions to implement droop volt-var + volt-watt control

In [ ]:
# changes to: my_heatmapSetup_funcs.py

In [ ]:
# changes to: my_detControlMatExistence_funcs.py

# indicMat is now 6n x 3n, [Fq Fp]'
def assignF(ver,Fp,Fq,indicMat): # algo similar to updateStateSpace
    
    n=int(len(indicMat)/6) # indicMat has 6n rows for all versions
    Hblock1=np.zeros((3*n,3*n))   
    ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) # python indexing goes first to (last-1)          
    for k in range(len(ridx)):
            Hblock1[ridx[k]][colidx[k]] = Fq
    
    Hblock2=np.zeros((3*n,3*n))   

    if ver==1: # PBC       
        ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) 
        for k in range(len(ridx)):
                Hblock2[ridx[k]][colidx[k]] = Fp

        upper=np.concatenate((Hblock1, np.zeros((3*n,3*n))),axis=1)
        lower=np.concatenate((np.zeros((3*n,3*n)),Hblock2),axis=1)
        F=np.concatenate((upper,lower))  # indicMat is 6n x 6n, [Fq 0 ; 0 Fp]
    
    else: # Droop:        
        ridx,colidx=np.nonzero(indicMat[3*n+1:,0:3*n]) 
        for k in range(len(ridx)):
                Hblock2[ridx[k]][colidx[k]] = Fp
        F=np.concatenate(Hblock1,Hblock2,axis=0) # indicMat is now 6n x 3n, [Fq Fp]'

    #print(F)
    print("Size of F=",F.shape)
    return F

def setupStateSpace(ver, n, feeder, node_index_map, depths):
    #initializes state space matrices A and B
    #n = number of nodes in network
    #feeder = initiaized feeder object
    #node_index_map = dictionary of node indices with node names as keys
    R, X = createRXmatrices_3ph(feeder, node_index_map,depths)
    concat_XR=np.concatenate((X, R), axis = 1)
    
    if ver==1: # PBC       
        A = np.identity(3*n)
        B = concat_XR # (6n*3n) matrix
    else: # volt-watt and volt-var
        A = np.identity(6*n)
        concat_XR_halfs = np.concatenate(((-1/2) * R, (1/2) * X), axis = 1)
        B = np.concatenate((concat_XR, concat_XR_halfs))
    
    return A, B


def updateStateSpace(ver,feeder, n, act_locs, perf_nodes,node_index_map):
    #creates (6n*3n) matrix with 1 at (3i+ph)(3j+ph) for volt-watt control, and (3i+3n+ph)(3j+ph) for volt-var control
    #in the above description, ph is the integer representation (a=0, b=1, c=2) of the phase intersection between the actuator and performance nodes
    #if an actuator and performance node have no phases in common, a warning is printed
    #n = number of nodes in network
    #act_locs = list of actuators locations in network (list of strings)
    #perf_nodes = list of performance nodes 
    #node_index_map = dictionary of node indices for indicMat and F matrix
    if ver==1: # PBC
        indicMat = np.zeros((6*n,6*n))
    else: # volt-watt and volt-var
        indicMat = np.zeros((6*n,3*n))
    
    # act_loc should now have string format of 'PBCbus_XXX', 'VVCbus_XXX', or 'VWCbus_XXX'
    for i in range(len(act_locs)): 
        act = act_locs[i]
        perf = perf_nodes[i]
        ctrlType=act[0:2] # 'PBC,'VVC', or 'VWC' type of control
        act_phases = feeder.busdict[act[7:]].phases # [7:] extracts the YYY bus number from 'XXXbus_YYY'
        perf_phases = feeder.busdict[perf[7:]].phases
        act_index = node_index_map[act]
        perf_index = node_index_map[perf]
        
        phase_intrsct = [ph for ph in act_phases if ph in perf_phases]
        if phase_intrsct == []: # disallow configs in which the act and perf node phases are not aligned. Results in kgain=0.0001 and thinks it's feasible
            print('WARNING: act_node ' + act + ' can NOT track perf_node ' + perf + ' --> no common phases')
            phase_loop_check=False
            break # if any actuator of the config has phase mismtach (between act and perf), don't evaluate the config
        else:
            phase_loop_check=True
            for i in range(len(phase_intrsct)):
                if phase_intrsct[i] == 'a':
                    phase_intrsct[i] = 0
                elif phase_intrsct[i] == 'b':
                    phase_intrsct[i] = 1
                elif phase_intrsct[i] == 'c':
                    phase_intrsct[i] = 2

        if ctrlType=='PBC':
            for ph in phase_intrsct:
                indicMat[(act_index*3)+ph][(perf_index*3)+ph] = 1
                indicMat[(act_index*3)+(3*n)+ph][(perf_index*3)+(3*n)+ph] = 1  
        elif ctrlType='VVC':
            for ph in phase_intrsct:
                indicMat[(act_index*3)+ph][(perf_index*3)+ph] = 1
        elif ctrlType='VWC': #volt-watt control
            for ph in phase_intrsct:
                indicMat[(act_index*3)+(3*n)+ph][(perf_index*3)+ph] = 1   
        else:
            except:
            print("actuator node first 3 char unrecognised for control type")
            
    return indicMat,phase_loop_check

In [ ]:
# detControlMatExistence(ver
# computeFeas_v1(ver)
# eval_config(ver,
# find_good_colocated(ver,
# runHeatMapProcess(ver,
# placeMaxColocActs_stopAtInfeas(ver,
# place_max_coloc_acts(ver,